The collaborative filter approach focuses on finding users who have given similar ratings to the same books, thus creating a link between users, to whom will be suggested books that were reviewed in a positive way. In this way, we look for associations between users, not between books.

In [99]:
import numpy as np
import pandas as pd
#import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

from joblib import dump

import tensorflow.compat.v1 as tf



In [100]:
rating = pd.read_csv('FoodRatings2.csv', sep=';', error_bad_lines=False, encoding="latin-1")
user = pd.read_csv('UserID.csv', sep=';', error_bad_lines=False, encoding="latin-1")
food = pd.read_csv('Food.csv', sep=';', error_bad_lines=False, encoding="latin-1")
food_rating = pd.merge(rating, food, on='foodid')
cols = ['UserID', 'FoodID', 'Rating', 'Food']
##food_rating.drop(cols, axis=1, inplace=True)
food_rating

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27788\2323556517.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  rating = pd.read_csv('FoodRatings2.csv', sep=';', error_bad_lines=False, encoding="latin-1")
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27788\2323556517.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  user = pd.read_csv('UserID.csv', sep=';', error_bad_lines=False, encoding="latin-1")
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27788\2323556517.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  food = pd.read_csv('Food.csv', sep=';', error_bad_lines=False, encoding="latin-1")


,userid,foodid,rating,food
0,99,9,2,Bee Hoon Goreng
1,99,9,2,Bee Hoon Goreng
2,98,9,2,Bee Hoon Goreng
3,95,9,3,Bee Hoon Goreng
4,92,9,4,Bee Hoon Goreng
...,...,...,...,...
7445,12,23,2,Curry chicken noodles
7446,11,23,5,Curry chicken noodles
7447,11,23,3,Curry chicken noodles
7448,11,23,3,Curry chicken noodles


In [101]:
rating_count = (food_rating.
     groupby(by = ['food'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'RatingCount_Food'})
     [['food', 'RatingCount_Food']])
rating_count

,food,RatingCount_Food
0,Assam Pedas,40
1,Ayam Penyet,74
2,Bak Kut Teh,80
3,Bak chor mee,76
4,Bakso,98
...,...,...
84,rawon,99
85,soup kambing,93
86,takoyaki,83
87,text,78


In [102]:
threshold = 10
rating_count = rating_count.query('RatingCount_Food >= @threshold')
rating_count.head()

,food,RatingCount_Food
0,Assam Pedas,40
1,Ayam Penyet,74
2,Bak Kut Teh,80
3,Bak chor mee,76
4,Bakso,98


In [103]:
user_rating = pd.merge(rating_count, food_rating, left_on='food', right_on='food', how='left')

In [104]:
user_rating.head()

,food,RatingCount_Food,userid,foodid,rating
0,Assam Pedas,40,99,1,4
1,Assam Pedas,40,97,1,3
2,Assam Pedas,40,84,1,4
3,Assam Pedas,40,83,1,3
4,Assam Pedas,40,83,1,2


In [105]:
user_count = (user_rating.
     groupby(by = ['userid'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'RatingCount_user'})
     [['userid', 'RatingCount_user']]
    )
user_count

,userid,RatingCount_user
0,1,33
1,2,86
2,3,69
3,4,68
4,5,69
...,...,...
96,97,79
97,98,86
98,99,89
99,100,28


In [106]:
threshold = 9
user_count = user_count.query('RatingCount_user >= @threshold')
user_count

,userid,RatingCount_user
0,1,33
1,2,86
2,3,69
3,4,68
4,5,69
...,...,...
96,97,79
97,98,86
98,99,89
99,100,28


In [107]:
combined = user_rating.merge(user_count, left_on = 'userid', right_on = 'userid', how = 'inner')

In [108]:
combined

,food,RatingCount_Food,userid,foodid,rating,RatingCount_user
0,Assam Pedas,40,99,1,4,89
1,Ayam Penyet,74,99,3,5,89
2,Bak Kut Teh,80,99,5,4,89
3,Bak Kut Teh,80,99,5,4,89
4,Bak Kut Teh,80,99,5,3,89
...,...,...,...,...,...,...
7445,takoyaki,83,1,80,2,33
7446,tom yum,79,1,85,4,33
7447,tom yum,79,1,85,4,33
7448,tom yum,79,1,85,4,33


In [109]:
combined.shape
combined.loc[combined['userid'] == 89]

,food,RatingCount_Food,userid,foodid,rating,RatingCount_user
2946,Ayam Penyet,74,89,3,3,61
2947,Bak Kut Teh,80,89,5,1,61
2948,Bakso,98,89,6,4,61
2949,Bakso,98,89,6,2,61
2950,BanMian,90,89,7,4,61
...,...,...,...,...,...,...
3002,oyster omelette,75,89,58,5,61
3003,rawon,99,89,65,2,61
3004,takoyaki,83,89,80,3,61
3005,takoyaki,83,89,80,2,61


In [110]:
print('Number of unique Food: ', combined['food'].nunique())
print('Number of unique Users: ', combined['userid'].nunique())

Number of unique Food:  89
Number of unique Users:  101


Normalize the ratings.

In [111]:
scaler = MinMaxScaler()
combined['rating'] = combined['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(combined['rating'].values.reshape(-1,1)))
combined['rating'] = rating_scaled
combined['rating']

0       0.75
1       1.00
2       0.75
3       0.75
4       0.50
        ... 
7445    0.25
7446    0.75
7447    0.75
7448    0.75
7449    0.75
Name: rating, Length: 7450, dtype: float64

Abd build the user book matrix.

In [112]:
combined = combined.drop_duplicates(['userid', 'food'])
user_food_matrix = combined.pivot(index='userid', columns='food', values='rating')
user_food_matrix.fillna(0, inplace=True)

users = user_food_matrix.index.tolist()
books = user_food_matrix.columns.tolist()

user_food_matrix = user_food_matrix.values

tf.placeholder only available in v1, so we have to work around. 

In [113]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

We will initialize the TensorFlow placeholder. Then, weights and biases are randomly initialized, the following code are taken from the book: Python Machine Learning Cook Book - Second Edition

In [114]:
num_input = combined['food'].nunique()
num_hidden_1 = 10
num_hidden_2 = 5

X = tf.placeholder(tf.float64, [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}

Now, we can build the encoder and decoder model, as follows:

In [115]:
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    return layer_2

We will construct the model and the predictions

In [116]:
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

y_pred = decoder_op

y_true = X

define loss function and optimizer, and minimize the squared error, and define the evaluation metrics

In [117]:
loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)
eval_x = tf.placeholder(tf.int32, )
eval_y = tf.placeholder(tf.int32, )
pre, pre_op = tf.metrics.precision(labels=eval_x, predictions=eval_y)

Initialize the variables. Because TensorFlow uses computational graphs for its operations, placeholders and variables must be initialized.

In [118]:
init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()
pred_data = pd.DataFrame()

We can finally start to train our model.

We split training data into batches, and we feed the network with them.

We train our model with vectors of user ratings, each vector represents a user and each column a book, and entries are ratings that the user gave to books. 

After a few trials, I discovered that training model for 5 epochs with a batch size of 10 would be consum enough memory. This means that the entire training set will feed our neural network 20 times, every time using 50 users.

In [119]:
#saver = tf.compat.v1.train.Saver()
with tf.Session() as session:
    epochs = 100
    batch_size = 35
    

    session.run(init)
    session.run(local_init)

    num_batches = int(user_food_matrix.shape[0] / batch_size)
    user_food_matrix = np.array_split(user_food_matrix, num_batches)
    
    for i in range(epochs):

        avg_cost = 0
        for batch in user_food_matrix:
            _, l = session.run([optimizer, loss], feed_dict={X: batch})
            avg_cost += l

        avg_cost /= num_batches

        print("epoch: {} Loss: {}".format(i + 1, avg_cost))
    
    # Save the final model
    #saver.save(session, 'model.ckpt')

    user_food_matrix = np.concatenate(user_food_matrix, axis=0)

    preds = session.run(decoder_op, feed_dict={X: user_food_matrix})

    pred_data = pred_data.append(pd.DataFrame(preds))

    pred_data = pred_data.stack().reset_index(name='rating')
    pred_data.columns = ['userid', 'food', 'rating']
    pred_data['userid'] = pred_data['userid'].map(lambda value: users[value])
    pred_data['food'] = pred_data['food'].map(lambda value: books[value])
    
    keys = ['userid', 'food']
    index_1 = pred_data.set_index(keys).index
    index_2 = combined.set_index(keys).index

    top_ten_ranked = pred_data[~index_1.isin(index_2)]
    top_ten_ranked = top_ten_ranked.sort_values(['userid', 'rating'], ascending=[True, False])
    top_ten_ranked = top_ten_ranked.groupby('userid').head(10)
    
        


epoch: 1 Loss: 0.27771639823913574
epoch: 2 Loss: 0.2776837646961212
epoch: 3 Loss: 0.27764759957790375
epoch: 4 Loss: 0.27760739624500275
epoch: 5 Loss: 0.2775627672672272
epoch: 6 Loss: 0.27751316130161285
epoch: 7 Loss: 0.2774581164121628
epoch: 8 Loss: 0.2773970067501068
epoch: 9 Loss: 0.27732911705970764
epoch: 10 Loss: 0.27725377678871155
epoch: 11 Loss: 0.2771701514720917
epoch: 12 Loss: 0.2770773321390152
epoch: 13 Loss: 0.27697426080703735
epoch: 14 Loss: 0.27685996890068054
epoch: 15 Loss: 0.2767331004142761
epoch: 16 Loss: 0.2765924036502838
epoch: 17 Loss: 0.27643631398677826
epoch: 18 Loss: 0.2762632369995117
epoch: 19 Loss: 0.27607133984565735
epoch: 20 Loss: 0.275858610868454
epoch: 21 Loss: 0.2756228744983673
epoch: 22 Loss: 0.2753617614507675
epoch: 23 Loss: 0.27507248520851135
epoch: 24 Loss: 0.27475228905677795
epoch: 25 Loss: 0.2743978351354599
epoch: 26 Loss: 0.2740057557821274
epoch: 27 Loss: 0.2735721617937088
epoch: 28 Loss: 0.2730928957462311
epoch: 29 Loss: 0.

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27788\1087785261.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_data = pred_data.append(pd.DataFrame(preds))


In [120]:
print(users)
print(pred_data['userid'].unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 123]
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 123]


In [51]:
top_ten_ranked.loc[top_ten_ranked['userid'] == 123]

,userid,food,rating
8914,123,Char Kway Teow,0.448946
8962,123,Satay,0.442345
8947,123,Naan,0.430778
8945,123,Mixed Grill,0.421203
8910,123,Big bao,0.410813
8919,123,Crab bee Hoon,0.405274
8909,123,Beef Noodle Soup,0.405114
8949,123,Nasi lemak,0.402376
8985,123,soup kambing,0.396846
8943,123,Mee siam,0.391838


In [68]:
    saver.save(session, 'model.ckpt')
    joblib.dump(users, 'users.joblib')
    joblib.dump(foods, 'foods.joblib')


RuntimeError: Attempted to use a closed Session.

In [22]:
#book_rating.loc[book_rating['User-ID'] == 278582].sort_values(by=['Book-Rating'], ascending=False)